In [4]:
# imports
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
load_dotenv()

api_key = os.getenv('FOURSQUARE_API_KEY')
location = 'London, United Kingdom'
url = "https://api.foursquare.com/v3/places/search?near=" + location

# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = api_key

result = requests.get(url, headers=headers)
london= json.loads(result.text)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
#I need to extract the most
#relevant data in our case. 

instance = london['results']
#1. I will loop through the dictionary and get the information i need
foursquare = []
for item in instance:
    foursquare.append({
        'distance':item['distance'],
        'name':item['name'],
        'location':item['location']
    })


Put your parsed results into a DataFrame

In [10]:

#3. I will parse foursquare into dataframe
foursquare_df = pd.DataFrame(foursquare)
foursquare_df

,distance,name,location
0,193,National Gallery,"{'address': 'Trafalgar Sq', 'admin_region': 'E..."
1,494,WatchHouse,"{'address': '7 Upper St Martins Lane', 'admin_..."
2,617,Picturehouse Central,"{'address': 'Shaftesbury Ave', 'admin_region':..."
3,683,Foyles,"{'address': '113-119 Charing Cross Rd', 'admin..."
4,856,International House London,"{'address': '16 Stukeley St', 'admin_region': ..."
5,873,Big Ben (Elizabeth Tower),"{'address': 'Parliament Sq', 'admin_region': '..."
6,1017,Palace of Westminster,"{'address': 'Houses of Parliament', 'admin_reg..."
7,1156,Scarfes Bar,"{'address': '252 High Holborn', 'admin_region'..."
8,1165,British Museum,"{'address': 'Great Russell St', 'admin_region'..."
9,1219,Dior,"{'address': '160-162 New Bond St', 'admin_regi..."


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [12]:
#In order to apply all the skills I learnt, I chose to send this API request using Postman. I saved the postman json as cvs and opened it here
with open('response.json', 'r') as f:
    yelp = json.loads(f.read())


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [13]:

# Extracting id, name, review_count, and rating from each dictionary in the list
data = []
for business in yelp['businesses']:
    data.append({
        'id':business['id'],
        'name': business['name'],
        'latitude': business['coordinates']['latitude'],
        'longitude': business['coordinates']['longitude'],
        'review_count': business['review_count'],
        'rating': business['rating'],
        'distance': business['distance'],
        'is_closed': business['is_closed']
    })




Put your parsed results into a DataFrame

In [84]:
# Converting the list of dictionaries into a DataFrame
yelp_d = pd.DataFrame(data, columns = ['name', 
                                     'review_count', 'rating', 'latitude', 'longitude' , 'is_closed', 'distance'])

#rename 'name' as 'resto_name' to avoid future confusion
yelp_df= yelp_d.rename(columns={'name': 'resto_name'})

# Displaying the DataFrame
yelp_df

,resto_name,review_count,rating,latitude,longitude,is_closed,distance
0,Dishoom,2556,4.5,51.512447,-0.126958,False,626.567432
1,Sketch,1031,4.0,51.512761,-0.141592,False,388.943719
2,Yauatcha,531,4.0,51.513708,-0.135129,False,104.027201
3,Dishoom,729,4.6,51.513087,-0.139767,False,262.732936
4,The Mayfair Chippy,510,4.5,51.513134,-0.153396,False,1205.537933
5,Blacklock,150,4.6,51.511687,-0.134762,False,162.359136
6,Kiln,82,4.3,51.511411,-0.136050,False,170.850239
7,Bob Bob Ricard,190,4.1,51.512310,-0.137234,False,111.345078
8,Flat Iron,480,4.5,51.512120,-0.138401,False,186.272039
9,Social Eating House,99,4.1,51.513673,-0.136388,False,86.721346


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp data provides more coverage compared to Foursquare data in this situation. Since I want to measure business performance accross the city, I will use reviews and ratings. So, Yelp API is better because it provides both number of revies and ratings per poi. 

Get the top 10 restaurants according to their rating

In [85]:
yelp_resto = yelp_df.sort_values(by=['rating'], ascending = False)
best_resto = yelp_resto.head(10)

best_resto

,resto_name,review_count,rating,latitude,longitude,is_closed,distance
17,The Ledbury,173,4.6,51.516819,-0.200107,False,4458.536864
3,Dishoom,729,4.6,51.513087,-0.139767,False,262.732936
5,Blacklock,150,4.6,51.511687,-0.134762,False,162.359136
0,Dishoom,2556,4.5,51.512447,-0.126958,False,626.567432
8,Flat Iron,480,4.5,51.512120,-0.138401,False,186.272039
4,The Mayfair Chippy,510,4.5,51.513134,-0.153396,False,1205.537933
15,Burger & Lobster,114,4.5,51.501441,-0.159432,False,2066.044802
13,Hawksmoor Seven Dials,398,4.4,51.513500,-0.125674,False,715.805011
10,Dinner by Heston Blumenthal,312,4.4,51.502163,-0.159966,False,2047.209028
19,Petrus,135,4.4,51.499367,-0.156520,False,2065.043045


In [87]:
#I'll export this for future use
best_resto.to_excel('restaurants.xlsx',sheet_name= 'restaurants', index=False)